In [ ]:

!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 11.6 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
from transformers import BlipForQuestionAnswering, BlipProcessor
from PIL import Image

# Load the model and processor
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

# Example image and question
image_path = "/content/sample.jpeg"  # Replace with your image path
image = Image.open(image_path)

question = "What is in the picture?"

# Prepare inputs
inputs = processor(image, question, return_tensors="pt")

# Generate the answer with controlled token length
output = model.generate(**inputs, max_new_tokens=30)
answer = processor.decode(output[0], skip_special_tokens=True)

print(f"Question: {question}")
print(f"Answer: {answer}")


Question: What is in the picture?
Answer: three women


In [ ]:
!pip install gtts


In [ ]:
import gradio as gr
from transformers import BlipForQuestionAnswering, BlipProcessor
from PIL import Image
from gtts import gTTS
import os

In [ ]:
from huggingface_hub import notebook_login

# Run this to log in
notebook_login()


In [ ]:
pip install transformers gradio pillow


In [ ]:
import gradio as gr
from transformers import BlipForQuestionAnswering, BlipProcessor
from PIL import Image
from gtts import gTTS

# Load the BLIP VQA model and processor
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

# Define the function for answering questions with optional voice output
def vqa_with_voice_toggle(image, question, enable_voice):
    # Generate the answer
    inputs = processor(image, question, return_tensors="pt")
    output = model.generate(**inputs, max_new_tokens=30)
    answer = processor.decode(output[0], skip_special_tokens=True)

    if enable_voice:
        # Convert answer to speech
        tts = gTTS(answer)
        audio_path = "answer.mp3"
        tts.save(audio_path)
        return answer, audio_path
    else:
        return answer, None

# Create the Gradio interface
interface = gr.Interface(
    fn=vqa_with_voice_toggle,
    inputs=[
        gr.Image(type="pil", label="Upload Image"),
        gr.Textbox(lines=1, placeholder="Ask a question about the image", label="Question"),
        gr.Checkbox(label="Enable Voice Output", value=True),
    ],
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Audio(label="Answer (Voice)", autoplay=True),
    ],
    title="Visual Question Answering with Voice Toggle",
    description="Upload an image, ask a question, and get the answer as text and optionally as autoplay voice output.",
)

# Launch the interface
interface.launch()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
import gradio as gr
from transformers import BlipForQuestionAnswering, BlipProcessor
from PIL import Image

# Load the BLIP VQA model and processor
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

# Global variable to store the uploaded image
uploaded_image = None

# Function to upload an image
def upload_image(image):
    global uploaded_image
    uploaded_image = image  # Store the image for subsequent questions
    return [("System", "Image uploaded successfully! You can now ask questions.")]

# Function to process a question and return an answer
def ask_question(history, question):
    if uploaded_image is None:
        return history + [("System", "Please upload an image first.")]

    # Generate the answer
    inputs = processor(uploaded_image, question, return_tensors="pt")
    output = model.generate(**inputs, max_new_tokens=30)
    answer = processor.decode(output[0], skip_special_tokens=True)

    # Append the question and answer to the conversation history
    history += [("User", question), ("System", answer)]
    return history

# Create Gradio chatbot interface
with gr.Blocks() as chatbot_interface:
    with gr.Row():
        gr.Markdown("## Visual Question Answering Chatbot")
    with gr.Row():
        gr.Markdown(
            "Upload an image, then ask multiple questions about the image. "
            "The answers will appear in a conversational flow."
        )
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Upload Image", interactive=True)
            upload_button = gr.Button("Upload Image")
            chat_history = gr.Chatbot(label="Chatbot")
            question_input = gr.Textbox(
                placeholder="Type your question here...",
                label="Ask a Question",
                lines=1,
            )
            send_button = gr.Button("Send")

    # Define interactions
    upload_button.click(
        upload_image,
        inputs=image_input,
        outputs=chat_history,
    )
    send_button.click(
        ask_question,
        inputs=[chat_history, question_input],
        outputs=chat_history,
    )

# Launch the chatbot
chatbot_interface.launch()


/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:237: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


In [ ]:
pip install SpeechRecognition


In [ ]:
!pip install pydub

from pydub import AudioSegment
import os
import speech_recognition as sr

In [ ]:
import gradio as gr
from transformers import BlipForQuestionAnswering, BlipProcessor
from PIL import Image
from gtts import gTTS
import speech_recognition as sr

# Load BLIP model and processor
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

# Function for speech-to-text conversion
# def speech_to_text(audio_file):
#     recognizer = sr.Recognizer()
#     with sr.AudioFile(audio_file) as source:
#         audio_data = recognizer.record(source)
#         return recognizer.recognize_google(audio_data)

def speech_to_text(audio_file):
    try:
        # Convert M4A to WAV if necessary
        if not audio_file.endswith(".wav"):
            audio = AudioSegment.from_file(audio_file, format="m4a")  # Specify format
            wav_file = "converted.wav"
            audio.export(wav_file, format="wav")
            audio_file = wav_file

        # Process the WAV file with speech_recognition
        recognizer = sr.Recognizer()
        with sr.AudioFile(audio_file) as source:
            audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data)

        # Clean up temporary WAV file
        if os.path.exists("converted.wav"):
            os.remove("converted.wav")

        return text
    except Exception as e:
        return f"Error: {str(e)}"

# Function for visual question answering
def vqa_with_voice_toggle(image, question, enable_voice):
    inputs = processor(image, question, return_tensors="pt")
    output = model.generate(**inputs, max_new_tokens=30)
    answer = processor.decode(output[0], skip_special_tokens=True)

    audio_path = None
    if enable_voice:
        tts = gTTS(answer)
        audio_path = "answer.mp3"
        tts.save(audio_path)
    return answer, audio_path

# Gradio interface
with gr.Blocks() as interface:
    gr.Markdown("## Visual Question Answering with Chatbot and Voice Support")
    with gr.Row():
        image_input = gr.Image(label="Upload Image", type="pil")
        chatbot = gr.Chatbot(label="Chat History", type="messages")

    with gr.Row():
        question_input = gr.Textbox(
            placeholder="Ask a question about the image",
            label="Type your question",
            lines=1,
        )
        voice_input = gr.Audio(label="Upload audio question", type="filepath")
        send_button = gr.Button("Send")
        enable_voice_output = gr.Checkbox(label="Enable Voice Output", value=True)

    with gr.Row():
        answer_audio = gr.Audio(label="Answer (Voice)")

    # Function to handle user interactions
    def handle_interaction(image, chatbot_history, question, voice, enable_voice):
        if voice:
            question = speech_to_text(voice)
            chatbot_history.append({"role": "user", "content": f"Voice input: {question}"})
        elif question:
            chatbot_history.append({"role": "user", "content": question})

        if image and question:
            answer, audio_path = vqa_with_voice_toggle(image, question, enable_voice)
            chatbot_history.append({"role": "assistant", "content": answer})
            return chatbot_history, audio_path
        else:
            return chatbot_history, None

    # Set button functionality
    send_button.click(
        handle_interaction,
        inputs=[image_input, chatbot, question_input, voice_input, enable_voice_output],
        outputs=[chatbot, answer_audio],
    )

interface.launch()


In [ ]:
import gradio as gr
from transformers import BlipForQuestionAnswering, BlipProcessor
from PIL import Image
from gtts import gTTS
import speech_recognition as sr
from pydub import AudioSegment
import os

# Load BLIP model and processor
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

# Function for speech-to-text conversion
def speech_to_text(audio_file):
    try:
        # Convert M4A to WAV if necessary
        if not audio_file.endswith(".wav"):
            audio = AudioSegment.from_file(audio_file, format="m4a")  # Specify format
            wav_file = "converted.wav"
            audio.export(wav_file, format="wav")
            audio_file = wav_file

        # Process the WAV file with speech_recognition
        recognizer = sr.Recognizer()
        with sr.AudioFile(audio_file) as source:
            audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data)

        # Clean up temporary WAV file
        if os.path.exists("converted.wav"):
            os.remove("converted.wav")

        return text
    except Exception as e:
        return f"Error: {str(e)}"

# Function for visual question answering
def vqa_with_voice_toggle(image, question, enable_voice):
    inputs = processor(image, question, return_tensors="pt")
    output = model.generate(**inputs, max_new_tokens=30)
    answer = processor.decode(output[0], skip_special_tokens=True)

    audio_path = None
    if enable_voice:
        tts = gTTS(answer)
        audio_path = "answer.mp3"
        tts.save(audio_path)
    return answer, audio_path

# Gradio interface
with gr.Blocks() as interface:
    gr.Markdown("## Visual Question Answering with Chatbot and Voice Support")
    with gr.Row():
        image_input = gr.Image(label="Upload Image", type="pil")
        chatbot = gr.Chatbot(label="Chat History", type="messages")

    with gr.Row():
        question_input = gr.Textbox(
            placeholder="Ask a question about the image",
            label="Type your question",
            lines=1,
        )
        voice_input = gr.Audio(label="Upload audio question", type="filepath")
        send_button = gr.Button("Send")
        enable_voice_output = gr.Checkbox(label="Enable Voice Output", value=True)

    with gr.Row():
        answer_audio = gr.Audio(label="Answer (Voice)", autoplay=True)

    # Function to handle user interactions
    def handle_interaction(image, chatbot_history, question, voice, enable_voice):
        if voice:
            question = speech_to_text(voice)
            chatbot_history.append({"role": "user", "content": f"Voice input: {question}"})
        elif question:
            chatbot_history.append({"role": "user", "content": question})

        if image and question:
            answer, audio_path = vqa_with_voice_toggle(image, question, enable_voice)
            chatbot_history.append({"role": "assistant", "content": answer})
            return chatbot_history, audio_path, ""  # Return empty string to reset the question input
        else:
            return chatbot_history, None, question  # If no image/question, return the question for retrying

    # Set button functionality
    send_button.click(
        handle_interaction,
        inputs=[image_input, chatbot, question_input, voice_input, enable_voice_output],
        outputs=[chatbot, answer_audio, question_input],
    )

interface.launch()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
import gradio as gr
from transformers import BlipForQuestionAnswering, BlipProcessor
from PIL import Image
from gtts import gTTS
import speech_recognition as sr
from pydub import AudioSegment
import os

# Load BLIP model and processor
model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

# Function for speech-to-text conversion
def speech_to_text(audio_file):
    try:
        # Convert M4A to WAV if necessary
        if not audio_file.endswith(".wav"):
            audio = AudioSegment.from_file(audio_file, format="m4a")  # Specify format
            wav_file = "converted.wav"
            audio.export(wav_file, format="wav")
            audio_file = wav_file

        # Process the WAV file with speech_recognition
        recognizer = sr.Recognizer()
        with sr.AudioFile(audio_file) as source:
            audio_data = recognizer.record(source)
            text = recognizer.recognize_google(audio_data)

        # Clean up temporary WAV file
        if os.path.exists("converted.wav"):
            os.remove("converted.wav")

        return text
    except Exception as e:
        return f"Error: {str(e)}"

# Function to generate clarification based on question
def generate_clarification(question):
    # Define some simple criteria for clarification based on question content
    if any(keyword in question.lower() for keyword in ["objects", "everything", "thing", "type", "what is this","here","in this","there"]):
        return "Could you specify what you mean in specific?"

    if len(question.split()) < 3:
        return "Could you please provide more details or rephrase your question?"

    return None  # No clarification needed if none of the criteria are met

# Function for visual question answering
def vqa_with_voice_toggle(image, question, enable_voice):
    # Check for clarification first
    clarification = generate_clarification(question)

    # If clarification is needed, return it as part of the conversation
    if clarification:
        return clarification, None  # Return the clarification prompt

    # If no clarification needed, proceed with VQA
    inputs = processor(image, question, return_tensors="pt")
    output = model.generate(**inputs, max_new_tokens=30)
    answer = processor.decode(output[0], skip_special_tokens=True)

    # Generate audio output if voice output is enabled
    audio_path = None
    if enable_voice:
        tts = gTTS(answer)
        audio_path = "answer.mp3"
        tts.save(audio_path)

    return answer, audio_path

# Gradio interface
with gr.Blocks() as interface:
    gr.Markdown("## Visual Question Answering with Chatbot and Voice Support")

    with gr.Row():
        image_input = gr.Image(label="Upload Image", type="pil")
        chatbot = gr.Chatbot(label="Chat History", type="messages")

    with gr.Row():
        question_input = gr.Textbox(
            placeholder="Ask a question about the image",
            label="Type your question",
            lines=1,
        )
        voice_input = gr.Audio(label="Upload audio question", type="filepath")
        send_button = gr.Button("Send")
        enable_voice_output = gr.Checkbox(label="Enable Voice Output", value=True)

    with gr.Row():
        answer_audio = gr.Audio(label="Answer (Voice)", autoplay=True)

    # Function to handle user interactions
    def handle_interaction(image, chatbot_history, question, voice, enable_voice):
        if voice:
            question = speech_to_text(voice)
            chatbot_history.append({"role": "user", "content": f"Voice input: {question}"})
        elif question:
            chatbot_history.append({"role": "user", "content": question})

        if image and question:
            # Get the answer or clarification from VQA
            answer, audio_path = vqa_with_voice_toggle(image, question, enable_voice)

            # If clarification is needed, append it to the history
            if "Do you mean" in answer:
                chatbot_history.append({"role": "assistant", "content": answer})
                return chatbot_history, None, question  # Wait for user response to clarification

            chatbot_history.append({"role": "assistant", "content": answer})
            return chatbot_history, audio_path, ""  # Return empty string to reset the question input
        else:
            return chatbot_history, None, question  # If no image/question, return the question for retrying

    # Set button functionality
    send_button.click(
        handle_interaction,
        inputs=[image_input, chatbot, question_input, voice_input, enable_voice_output],
        outputs=[chatbot, answer_audio, question_input],
    )

interface.launch()


**Tourist Guide**





In [ ]:
# from transformers import BlipForQuestionAnswering, BlipProcessor

# model_name = "liuhaotian/llava-v1"

# model = BlipForQuestionAnswering.from_pretrained(model_name, use_auth_token=token)
# processor = BlipProcessor.from_pretrained(model_name, use_auth_token=token)


from huggingface_hub import notebook_login

# Run this to log in
notebook_login()


In [ ]:
!pip install transformers
!pip install torch
!pip install pillow


In [ ]:
!pip install git+https://github.com/LLaVA-VL/LLaVA-NeXT.git
!pip install torch decord numpy


In [ ]:
!pip install flash-attn
!pip install av


In [ ]:
!pip install huggingface_hub


In [ ]:
from huggingface_hub import hf_hub_download


In [ ]:
import av
import torch
import numpy as np
from transformers import VideoLlavaForConditionalGeneration, VideoLlavaProcessor

def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])

# Load the model in half-precision
model = VideoLlavaForConditionalGeneration.from_pretrained("LanguageBind/Video-LLaVA-7B-hf", torch_dtype=torch.float16, device_map="auto")
processor = VideoLlavaProcessor.from_pretrained("LanguageBind/Video-LLaVA-7B-hf")

# Load the video as an np.arrau, sampling uniformly 8 frames
video_path = video_path = "/content/sample_video.mp4"

container = av.open(video_path)
total_frames = container.streams.video[0].frames
indices = np.arange(0, total_frames, total_frames / 8).astype(int)
video = read_video_pyav(container, indices)

# For better results, we recommend to prompt the model in the following format
prompt = "USER: <video>\nWhat is the weather? ASSISTANT:"
inputs = processor(text=prompt, videos=video, return_tensors="pt")

out = model.generate(**inputs, max_new_tokens=60)
processor.batch_decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Expanding inputs for image tokens in Video-LLaVa should be done in processing. Please add `patch_size` and `vision_feature_select_strategy` to the model's processing config or set directly with `processor.patch_size = {{patch_size}}` and processor.vision_feature_select_strategy = {{vision_feature_select_strategy}}`. Using processors without these attributes in the config is deprecated and will throw an error in v4.44.
Expanding inputs for image tokens in Video-LLaVa should be done in processing. Please add `patch_size` and `vision_feature_select_strategy` to the model's processing config or set directly with `processor.patch_size = {{patch_size}}` and processor.vision_feature_select_strategy = {{vision_feature_select_strategy}}`. Using processors without these attributes in the config is deprecated and will throw an error in v4.47.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-ef403b6e9a40>", line 42, in <cell line: 42>
    out = model.generate(**inputs, max_new_tokens=60)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py", line 2215, in generate
    result = self._sample(
  File "/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py", line 3206, in _sample
    outputs = self(**model_inputs, return_dict=True)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1747, in _call_

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-ef403b6e9a40>", line 42, in <cell line: 42>
    out = model.generate(**inputs, max_new_tokens=60)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/_contextlib.py", line 116, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py", line 2215, in generate
    result = self._sample(
  File "/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py", line 3206, in _sample
    outputs = self(**model_inputs, return_dict=True)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    return self._call_impl(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py", line 1747, in _call_

TypeError: object of type 'NoneType' has no len()

In [ ]:
from huggingface_hub import hf_hub_download

dataset_url = "raushan-testing-hf/videos-test"
video_filename = "sample_video.mp4"

try:
    video_path = hf_hub_download(dataset_url, filename=video_filename)
except Exception as e:
    print(f"Error: {e}")
